<!-- Assignment 4 - WS 2021 -->

# Adaptive Optimisation (15 points)

This notebook contains the fourth assignment for the exercises in Deep Learning and Neural Nets 1.
It provides a skeleton, i.e. code with gaps, that will be filled out by you in different exercises.
All exercise descriptions are visually annotated by a vertical bar on the left and some extra indentation,
unless you already messed with your jupyter notebook configuration.
Any questions that are not part of the exercise statement do not need to be answered,
but should rather be interpreted as triggers to guide your thought process.

**Note**: The cells in the introductory part (before the first subtitle)
perform all necessary imports and provide utility function that should work without problems.
Please, do not alter this code or add extra import statements in your submission, unless it is explicitly requested!

<span style="color:#d95c4c">**IMPORTANT:**</span> Please, change the name of your submission file so that it contains your student ID!

In this assignment, the goal is to get familiar with some of the most common (adaptive) **optimisation algorithms**. Essentially, the most common optimisation algorithms are nothing more than variants of gradient descent. Although it is often claimed that stochastic gradient descent outperforms any adaptive learning method when carefully configured, it is often more convenient to use a method that requires less tuning.

In [1]:
import numpy as np
from matplotlib import pyplot as plt

from nnumpy import Module
from nnumpy.data import get_mnist_data
from nnumpy.utils import split_data, to_one_hot

%matplotlib inline

In [2]:
def accuracy(logits, labels):
    """
    Compute accuracy for classification network.
    
    Parameters
    ----------
    logits : ndarray
        The logit-predictions from the network.
    labels : ndarray
        The target labels for the task.
    
    Returns
    -------
    acc : float
        The fraction of correctly classified samples.
    """
    idx = np.argmax(logits, axis=1)
    return np.mean(labels[np.arange(len(idx)), idx])

In [3]:
class Optimiser:
    """ Base class for NNumpy optimisers. """

    def __init__(self, parameters, lr: float):
        """
        Create an optimiser instance.

        Parameters
        ----------
        parameters : iterable
            Iterable over the parameters that need to be updated.
        lr : float
            Learning rate or step size for updating the parameters.
        states : iterable
            Iterable over initial update states for each parameter.
        """
        self.parameters = list(parameters)
        if len(self.parameters) == 0:
            raise ValueError("no parameters to optimise")

        self.lr = float(lr)
        
        self._states = [self.init_state(par) for par in self.parameters]
    
    def init_state(self, par):
        """
        Create the initial optimiser state for a parameter.
        
        Parameters
        ----------
        par : Parameter
            The parameter to create the initial state for.
            
        Returns
        -------
        state : object
            The initial optimiser state for the given parameter.
        """
        return None

    def step(self):
        """
        Update all parameters under control of this optimiser
        by making one step in the direction computed by the algorithm
        for each of the parameters.
        """
        for par, state in zip(self.parameters, self._states):
            update = self.get_update(par, state)
            par -= self.lr * update
            del par.grad  # optional safeguard
    
    def get_update(self, par, state):
        """
        Compute the update for a single parameter.
        
        Parameters
        ----------
        par : Parameter
            The parameter (with `.grad` field) to compute the update for.
        state : object
            The state of the optimiser associated with the parameter.
        
        Returns
        -------
        
        """
        raise NotImplementedError("method must be implemented in subclass")

## Gradient Descent

As you should know by now, the *backpropagation* algorithm is little more than a combination of the chain rule and some form of gradient descent. Although this happens to work well in practice, it is good to be aware of possible issues when using first-order optimisation methods:

 1. Gradient descent can/will get stuck in *local minima*.
 2. The *gradient magnitude* tells you nothing about how far away minima are.
 3. When optimising the *empirical error*, gradient descent would require the gradient over the entire dataset.

Note that this last point is not necessarily an issue, but it is useful to keep in mind. Also, it implies that the gradients that can be computed on the entire dataset do not need to correspond to the gradients that would be required to minimise the generalisation error.

### Exercise 1: Stochasticity (2 Points)

Rather than using plain gradient descent algorithm, a stochastic variant is used to train neural networks. This variant is known as *Stochastic Gradient Descent*, or *SGD* for short. Although this naming scheme seems to suggest that stochasticity is part of the algorithm, it is actually introduced by how we use the data to compute gradients.

Instead of computing the gradients over the entire dataset in one go, the samples in the dataset are split up in more manageable pieces called *mini-batches*. This can speed up the computations significantly and avoids memory problems with very large datasets. Another benefit from mini-batches is that they introduce variation, or *stochasticity*, in the gradient computations. After all, the gradient for each mini-batch will be different to the gradient for other mini-batches or for all samples. This stochasticity can be useful to escape local minima in the optimisation process. To amplify this stochasticity, it is also common to shuffle the samples in the dataset so that mini-batches consist of different samples.

> Complete the `Dataloader` class below to process the data in mini-batches of pre-specified size. Also make sure to shuffle the data to get more stochasticity.

##### Some Notes on python generators

In python, a [generator](https://wiki.python.org/moin/Generators) is a function with some state that can return multiple values. You probably have already used generators without realising it. Probably, the most famous generator is `range`, which could be defined as follows:
```python
def _range(start, stop, step=1):
    i = start
    while i < stop:
        yield i
        i += step
```

Notice the `yield` keyword. This has a similar effect as `return` in that it provides a value to the outer scope of the function. However, it does not cause the function to be exited. Instead, the current state in the function is stored until the next value is requested. To get the return values of a generator, there are essentially two options:
 1. Using the `next` function. This will simply run the function until the next `yield` statement and give back the yielded value.
 2. By iterating over the generator in any way. This will consequently call `next` on the generator until the function exits.
 
For more information, please refer to the internet.

In [4]:
class Dataloader:
    
    def __init__(self, x, y, batch_size=None, shuffle=False, seed=None):
        """
        Parameters
        ----------
        x : (N, ...) ndarray
            the `N` input samples in the dataset
        y : (N, ...) ndarray
            the `N` output samples in the dataset
        batch_size : int, optional
            number of samples to include in a single mini-batch.
        shuffle : bool, optional
            whether or not the data should be shuffled.
        seed : int, optional
            seed for the pseudo random number generator used for shuffling.
        """
        self.x = x
        self.y = y
        self.batch_size = len(x) if batch_size is None else int(batch_size)
        self.shuffle = shuffle
        self.rng = np.random.default_rng(seed)
    
    def __iter__(self):
        """
        Iterates over the samples of the data.
        
        Yields
        ------
        x : ndarray
            input features for the batch
        y : ndarray
            target values for the batch
            
        Notes
        -----
        Each batch should contain the specified number of samples,
        except for the last batch if the batch_size does 
        not divide the number of samples in the data.
        """
        if self.shuffle:
            s_index = np.arange(0, len(self.x))
            self.rng.shuffle(s_index)
            self.x = self.x[s_index]
            self.y = self.y[s_index]
        
        i = self.batch_size
        j = 0

        while i <= len(self.x):
            yield self.x[j:i], self.y[j:i]
            
            j = i
            i += self.batch_size
            
        if j < len(self.x):
            yield self.x[j:], self.y[j:]
            
    def __len__(self):
        return 1 + (len(self.x) - 1) // self.batch_size

In [5]:
# sanity check
data_loader = Dataloader(np.arange(7), np.arange(7), batch_size=3, shuffle=True)
print(len(data_loader))
for x, y in data_loader:
    print(x.shape, y.shape)
print(x, y)

3
(3,) (3,)
(3,) (3,)
(1,) (1,)
[5] [5]


### Exercise 2: Gradient Descent with Momentum (2 Points)

Another way to keep gradient descent from getting stuck in local minima is to use momentum. Momentum accumulates the gradient directions over different batches and accelerates/decelarates the descent when all gradients point in the same/different direction(s). This also means that the update does not directly use the magnitude of the gradient, but instead focuses on the direction. 

> Implement the `get_update` method for the gradient descent optimiser with momentum.

**Hint:** you can use in-place operations to change numpy arrays in the state

In [6]:
class GradientDescent(Optimiser):
    """ NNumpy implementation of gradient descent. """

    def __init__(self, parameters, lr: float, momentum: float = 0.):
        """
        Parameters
        ----------
        momentum : float
            Momentum term for the gradient descent.
        """
        super().__init__(parameters, lr)
        self.mu = momentum

    def init_state(self, par):
        state = np.zeros_like(par)
        return state
        
    def get_update(self, par, state):
        grad = par.grad
        state = self.mu * state + (1 - self.mu) * grad
        par = par - self.lr * state

### Exercise 3: The Adam Optimiser (4 Points)

Momentum already provides a way to reduce the importance of the gradient magnitude. With adaptive learning rate methods, an attempt is made to ignore most of the magnitude information and the size of the update is mainly controlled by controlling the learning rate. One of the most popular first order adaptive methods in practice, is the Adam optimiser.

> Implement the `get_update` method for the Adam optimisation algorithm.

**Hint:** you can use in-place operations to change numpy arrays in the state

In [7]:
class Adam(Optimiser):
    """ NNumpy implementation of the Adam algorithm. """

    def __init__(self, parameters, lr: float = 1e-3, betas: tuple = (.9, .999),
                 epsilon: float = 1e-7, bias_correction=True):
        """
        Parameters
        ----------
        betas : tuple of 2 floats, optional
            Decay factors for the exponential averaging of mean, resp. variance.
        epsilon : float, optional
            Small number that is added to denominator for numerical stability.
        bias_correction : bool, optional
            Whether or not mean and bias estimates should be bias-corrected.
        """
        super().__init__(parameters, lr)

        beta1, beta2 = betas
        self.beta1 = float(beta1)
        self.beta2 = float(beta2)
        self.eps = float(epsilon)
        self.bias_correction = bias_correction

    def init_state(self, par):
#         state = [np.zeros_like(par), np.zeros_like(par), 1]
        
        return {'m': np.zeros_like(par), 'v': np.zeros_like(par), 'time_step': 1}

    def get_update(self, par, state):
#         grad = par.grad
#         m,v = state
#         m = self.beta1 * m + (1-self.beta1) * grad
#         v = self.beta2 * v + (1-self.beta2) * grad * grad
#         return par

#         state[0] += self.beta1
#         state[0] += (1-self.beta1) * par.grad
        
#         state[1] += self.beta2
#         state[1] += (1-self.beta2) * np.sqrt(par.grad)**2 #np.sqrt(par.grad)
        
#         if self.bias_correction:
#             m_hat = state[0] / (1 - self.beta1 ** state[2])
#             v_hat = state[1] / (1 - self.beta2 ** state[2])
            
#             state[2] += 1
            
#             return m_hat / (np.sqrt(v_hat) + self.eps)
#         else: 
#             return state[0] / (np.sqrt(state[1]) + self.eps)
        state['m'] = self.beta1*state['m'] + (1 - self.beta1)*par.grad
        state['v'] = self.beta2*state['v'] + (1 - self.beta2)*(par.grad**2)
        
        if self.bias_correction:
            m_hat = state['m'] / (1 - self.beta1**state['time_step'])
            v_hat = state['v'] / (1 - self.beta2**state['time_step'])
            update = (1/(np.sqrt(v_hat) + self.eps)) * m_hat
        else:
            update = (1/(np.sqrt(state['v']) + self.eps)) * state['m']
            
        state['time_step'] += 1
        
        return update

### Exercise 4: Evaluation and Update (3 Points)

When using the optimisers to fit a neural network to a given set of data, we can effectively minimise the empirical error. However, we are actually interested in minimising the risk. Therefore, it is also useful to evaluate the network regularly on unseen data.

 > Implement the `evaluate`, and `update` functions so that they perform the training and evaluation computations, respectively, for one iteration (aka *epoch*) over the entire dataset. Make sure to return the loss values to get loss curves at the end of this assignment.
 
**Hint:** you can use the `step` function of the optimiser to update the parameters.

In [27]:
def evaluate(network, metric, data_loader):
    """
    Evaluate a network by computing a metric for specific data.
    
    Parameters
    ----------
    network : Module
        A module that implements the network.
    metric : callable
        A function that takes logits and labels 
        and returns a scalar numpy array.
    data_loader : Dataloader
        The data loader that provides the batches.
        
    Returns
    -------
    values : ndarray
        The computed metric values for each batch in the data loader.
    """
    network.eval()
    errors = []
    
    for x_batch, y_batch in data_loader:
        forward = network.forward(x_batch)
        error = metric(forward,y_batch)
        errors.append(error)
        
    return np.array(errors)

#         values = []
#     for x,y in data_loader:
#         values.append(metric(network(x), y))
        
#     return np.array(values)


def update(network, loss, data_loader, optimiser):
    """
    Update a network by optimising the loss for the given data.
    
    Parameters
    ----------
    network : Module
        A module that implements the network.
    loss : Module
        Loss function module.
    data_loader : Dataloader
        The data loader that provides the batches.
    optimiser : Optimiser
        Optimisation algorithm to use for the update.
        
    Returns
    -------
    errors : ndarray
        The computed loss for each batch in the data loader.
    """
    network.train()
    loss.train()
    errors = []
    
    
    for x_batch, y_batch in data_loader:
        pred = network.forward(x_batch)
        error = loss.forward(pred,y_batch)
        network.zero_grad()
        grad = loss.backward()[0] #(forward - y_batch) #backprop of softmax & cross entropy
        network.backward(grad)
        print(error)
        optimiser.step()
        errors.append(np.mean(error))

#         pred = network(x)
#         print(pred)
#         l = loss.forward(pred, y)
#         print(l)
#         network.zero_grad()
#         l_grad = loss.backward()[0]
#         network.backward(l_grad)
#         optimiser.step()
#         errors.append(l)        
        
    return np.array(errors)


In [28]:
def train(train_data, valid_data, network, loss, optimiser, 
          epochs=1, batch_size=None, shuffle=True):
    """
    Train a neural network with gradient descent.
    
    Parameters
    ----------
    train_data : tuple of ndarrays
        Training data as tuple of input features and target values.
    valid_data : tuple of ndarrays
        Validation data as tuple of input features and target values.
    network : Module
        A module that implements the network.
    loss : Module
        Loss function module.
    optimiser : Optimiser
        Optimisation algorithm.
    epochs : int, optional
        Number of times to iterate the dataset.
    batch_size : int or None, optional
        Number of samples to use simultaneously.
        If None, all samples are fed to the network.
    shuffle : bool, optional
        Flag to enable or disable shuffling of the training data.
        
    Returns
    -------
    train_errors : (epochs + 1, n_batches) ndarray
        Training error for each epoch and each batch.
    valid_errors : (epochs + 1, 1) ndarray
        Validation error for each epoch.
    """
    train_loader = Dataloader(*train_data, batch_size=batch_size, shuffle=shuffle)
    valid_loader = Dataloader(*valid_data, batch_size=None)
    
    # log performance before training (for reference)
    train_errors = [evaluate(network, loss.eval(), train_loader)]
    valid_errors = [evaluate(network, loss.eval(), valid_loader)]
    # train for given number of epochs
    for _ in range(epochs):
        train_errors.append(update(network, loss, train_loader, optimiser))
        valid_errors.append(evaluate(network, loss.eval(), valid_loader))
        
    return np.stack(train_errors), np.stack(valid_errors)

### Exercise 5: Training a Neural Network (4 Points)

At this point, we have all ingredients to train fully connected and/or convolutional networks. Time to put your framework to the test and train a neural network to recognise hand-written digits.

> Use one of the optimisers from above to train a convolutional neural network on the MNIST dataset. You can use the `get_mnist_data` to download/process the data for this exercise. You will also need to create a neural network, for which you can make use of all modules from previous assignments. You are also allowed to create new modules and try out new things of course (make sure to include any new code in the notebook!). Achieve a model with 80% accuracy to collect all points. For reference: the sample solution (using the LeNet architecture illustrated below) takes &approx;15&nbsp;min to train for 10&nbsp;epochs.

**Hint:** Start with small models on a subset of the data. This will save you valuable time when trying out different settings!

<figure>
    <figcaption style="width: 100%; text-align: center;">LeNet-5 architecture</figcaption>
    <img src="https://miro.medium.com/max/2154/1*1TI1aGBZ4dybR6__DI9dzA.png" />
</figure>

In [29]:
# import the modules you have written in the previous assignment
# or alternatively, the code for these modules, here
# from nnumpy.__init__ import Sequential, Linear, Conv2D, Tanh, LogitCrossEntropy
from nnumpy import Module, Container, LossFunction
# from nnumpy.layers import Sequential, Linear, Conv2D, LogitCrossEntropy, AvgPool2d
from nnumpy.utils import sig2col

In [30]:
# you can create additional modules below

class Flatten(Module):
    """ NNumpy module to convert multi-dimensional outputs to a single vector. """
    
    def compute_outputs(self, x):
        return x.reshape(len(x), -1), x.shape
    
    def compute_grads(self, grads, shape):
        return grads.reshape(shape)

In [31]:
class Linear(Module):
    def __init__(self, in_features, out_features, use_bias=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.use_bias = use_bias
        
        # register parameters 'w' and 'b' here (mind use_bias!)
        self.register_parameter('w', np.empty((out_features,in_features)))
        if self.use_bias:
            self.register_parameter('b', np.empty(out_features))
        
        self.reset_parameters()
        
    def reset_parameters(self):
        self.w = np.random.randn(*self.w.shape)
        if self.use_bias:
            self.b = np.random.randn(*self.b.shape)
    
    def compute_outputs(self, x):
        self.x = x
        if self.use_bias:
            s = np.dot(self.x, self.w.T) + self.b
        else:
            s = np.dot(self.x, self.w.T)
        cache = x #(self.x, self.w) 
        
        return s, cache
    
    def compute_grads(self, grads, cache):
        x = cache
        dx = np.dot(grads, self.w)
        dw = np.dot(grads.T, x)
        db = np.sum(grads, axis=0)
        self.w.grad = dw
        self.b.grad = db
        
        return dx

class Sequential(Container):
    def __init__(self, *modules):
        super().__init__()
        if len(modules) == 1 and hasattr(modules[0], '__iter__'):
            modules = modules[0]
        
        for i, mod in enumerate(modules):
            self.add_module(mod)

    def compute_outputs(self, x):
        cache = x
        xx = x        
        for module in self._modules:
            y = module.forward(xx) 
            xx = y            
            
        return y, cache 
        
    def compute_grads(self, grads, cache):
        x = cache
        prev = grads
        
        for module in reversed(self._modules):
            module.zero_grad()
            dx = module.backward(prev)
            prev = dx
            
        return dx

class LogitCrossEntropy(LossFunction):
    def compute_outputs(self, logits, targets):
        e = np.exp(logits - np.max(logits,axis=1, keepdims=True))
        softmax = e/np.sum(e, axis=1, keepdims=True)        
        cross_entropy = np.sum(targets * np.log(softmax), axis=1, keepdims=True)
        cache = targets, softmax

        return cross_entropy, cache
            
    def compute_grads(self, grads, cache):
        targets, softmax = cache 
        dlogits = targets - softmax
        dtargets = np.log(softmax)
        
        return dlogits, dtargets

def multi_channel_convolution2d(x, k):
    print('x',x.shape)
    print(k.shape)
    x_view = sig2col(x,k.shape[-2:])
    return np.einsum('ncabij,ocij->noab',x_view,k)

class Conv2D(Module):
    def __init__(self, in_channels, out_channels, kernel_size, use_bias=True):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.use_bias = use_bias
        
        # register parameters 'w' and 'b' here (mind use_bias!)
        self.w = self.register_parameter('w', np.empty((out_channels, in_channels, *kernel_size)))
        self.b = self.register_parameter('b', np.empty((1,out_channels,1,1) if self.use_bias else 0))
        
        self.reset_parameters()
        
    def reset_parameters(self):
        self.w = np.random.randn(*self.w.shape)
        if self.use_bias:
            self.b = np.random.randn(*self.b.shape)
        
    def compute_outputs(self, x):
        N,Ci,H,W = x.shape
        if self.use_bias:
            feature_maps = multi_channel_convolution2d(x,self.w) + self.b
        else:
            feature_maps = multi_channel_convolution2d(x,self.w)
        cache = x
        return (feature_maps, cache)
            
    def compute_grads(self, grads, cache):
        x = cache               
        x = np.swapaxes(x,0,1)
        grads = np.swapaxes(grads,0,1)
        self.w.grad = np.swapaxes(self.w.grad, 0, 1)
        self.w.grad = np.swapaxes(multi_channel_convolution2d(x,grads),0,1)
        grads = np.swapaxes(grads,0,1)
        p1 = self.kernel_size[0] - 1
        p2 = self.kernel_size[1] - 1
        grads = np.pad(grads,((0,0),(0,0),(p1,p1),(p2,p2)))
        w = np.flip(np.flip(self.w,2),3) #(2,3))
        w = np.swapaxes(w,1,0)
        dx = multi_channel_convolution2d(grads, w)#np.swapaxes(self.w, 0, 1)) #np.moveaxis(self.w, 0, 1))

        return dx


class AvgPool2d(Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.kernel_size = tuple(kernel_size)
        
    def compute_outputs(self, x):
        w = np.full(shape=(self.kernel_size), fill_value=1/(self.kernel_size[0]*self.kernel_size[1]), dtype=np.float64)
        a = sig2col(x, w.shape, stride=self.kernel_size)
        a = a.reshape(*a.shape[:4],self.kernel_size[0]*self.kernel_size[1])
        a = np.mean(a, axis=-1)
        cache = (x,w)    
        return a, cache
    
    def compute_grads(self, grads, cache):
        (x,w) =  cache
        dx = np.zeros_like(x)
        for n in range(grads.shape[0]):
            i = 0
            for c in range(grads.shape[1]): 
                i = grads[n][c]/(w.shape[0]*w.shape[1])
                dx[n][c] = np.tile(i, (int(x.shape[2]/grads.shape[2]),int(x.shape[3]/grads.shape[3]))) 
        
        return dx

class Tanh(Module):        
    def compute_outputs(self, s):
        a = np.tanh(s)
        cache = s
        
        return a, cache
    
    def compute_grads(self, grads, cache):
        s = cache
        ds = grads * (1 - np.tanh(s)**2)
        
        return ds
    
class Tanh(Module):
    def compute_outputs(self, s):
        a = np.tanh(s)
        return a, a
    def compute_grads(self, grads, cache):
        return grads * (1 - cache**2)

In [32]:
def process_data(x_train, y_train):
    x_train = x_train.reshape(-1, 1, 28, 28)
    x_train = np.pad(x_train, ((0, 0), (0, 0), (2, 2), (2, 2)))
    y_train = to_one_hot(y_train)
    return x_train, y_train

x_train, y_train = get_mnist_data()
x_train, y_train = process_data(x_train[0:100], y_train[0:100])
print(x_train.shape, y_train.shape)

(100, 1, 32, 32) (100, 10)


In [33]:
C1 = Conv2D(in_channels=1, out_channels=6, kernel_size=(5,5), use_bias=True)
C1_tanh = Tanh()
S2 = AvgPool2d(kernel_size=(2,2))
C3 = Conv2D(in_channels=6, out_channels=16, kernel_size=(5,5), use_bias=True)
C3_tanh = Tanh()
S4 = AvgPool2d(kernel_size=(2,2))
C5 = Conv2D(in_channels=16, out_channels=120, kernel_size=(5,5), use_bias=True)
C5_tanh = Tanh()
flatten = Flatten()
F6 = Linear(in_features=120, out_features=84, use_bias=True)
F6_tanh = Tanh()
O = Linear(in_features=84, out_features=10, use_bias=True)

cnn = Sequential(C1,C1_tanh,S2,C3,C3_tanh,S4,C5,C5_tanh,flatten,F6,F6_tanh,O)
cnn.reset_parameters()


In [34]:
optimiser = Adam(cnn.parameters(), lr=0.001, bias_correction=True)

loss = LogitCrossEntropy()

ratio = 0.9
data_len = (len(x_train))
train_data, valid_data = (x_train[:int(ratio*data_len)] ,y_train[:int(ratio*data_len)]),(x_train[int(ratio*data_len):] ,y_train[int(ratio*data_len):] )

train_err, valid_err = train(train_data, valid_data, network=cnn, loss=loss, optimiser=optimiser, batch_size=10, shuffle=True, epochs=10)

x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6,

x (6, 10, 14, 14)
(16, 10, 10, 10)
x (10, 16, 18, 18)
(6, 16, 5, 5)
x (1, 10, 32, 32)
(6, 10, 28, 28)
x (10, 6, 36, 36)
(1, 6, 5, 5)
[[-14.71612647]
 [ -9.74163847]
 [-19.98957896]
 [-13.46195844]
 [-11.72513016]
 [-19.68754689]
 [ -0.20282496]
 [-14.91859712]
 [ -1.56792242]
 [-10.51040627]]
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (16, 10, 5, 5)
(120, 10, 1, 1)
x (10, 120, 9, 9)
(16, 120, 5, 5)
x (6, 10, 14, 14)
(16, 10, 10, 10)
x (10, 16, 18, 18)
(6, 16, 5, 5)
x (1, 10, 32, 32)
(6, 10, 28, 28)
x (10, 6, 36, 36)
(1, 6, 5, 5)
[[-11.09579045]
 [-14.71612647]
 [-28.25565117]
 [-13.00328135]
 [-18.47538662]
 [ -0.09271216]
 [-14.69166039]
 [ -3.86517487]
 [ -6.86273895]
 [-11.5159834 ]]
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (16, 10, 5, 5)
(120, 10, 1, 1)
x (10, 120, 9, 9)
(16, 1

x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (16, 10, 5, 5)
(120, 10, 1, 1)
x (10, 120, 9, 9)
(16, 120, 5, 5)
x (6, 10, 14, 14)
(16, 10, 10, 10)
x (10, 16, 18, 18)
(6, 16, 5, 5)
x (1, 10, 32, 32)
(6, 10, 28, 28)
x (10, 6, 36, 36)
(1, 6, 5, 5)
[[ -3.62245433]
 [ -3.86517487]
 [-12.24180784]
 [ -4.33989053]
 [ -7.3001145 ]
 [-29.41990522]
 [-17.10893197]
 [-11.09579045]
 [-11.66182547]
 [ -6.86273895]]
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (16, 10, 5, 5)
(120, 10, 1, 1)
x (10, 120, 9, 9)
(16, 120, 5, 5)
x (6, 10, 14, 14)
(16, 10, 10, 10)
x (10, 16, 18, 18)
(6, 16, 5, 5)
x (1, 10, 32, 32)
(6, 10, 28, 28)
x (10, 6, 36, 36)
(1, 6, 5, 5)
[[ -0.10960237]
 [ -1.56792242]
 [-11.16980434]
 [ -2.18944944]
 [ -9.85900002]
 [ -4.25626459]
 [-21.01344854]
 [-13.20524645]
 [-14.5405211 ]
 [ -1.3898454 ]]
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (16, 10, 5, 5)
(120

[[-15.22096192]
 [ -6.22033995]
 [-22.28898848]
 [ -1.3898454 ]
 [-29.41990522]
 [-17.37312804]
 [-14.81696756]
 [ -6.22764936]
 [ -4.25626459]
 [-21.01344854]]
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (16, 10, 5, 5)
(120, 10, 1, 1)
x (10, 120, 9, 9)
(16, 120, 5, 5)
x (6, 10, 14, 14)
(16, 10, 10, 10)
x (10, 16, 18, 18)
(6, 16, 5, 5)
x (1, 10, 32, 32)
(6, 10, 28, 28)
x (10, 6, 36, 36)
(1, 6, 5, 5)
[[-15.52044971]
 [-12.24180784]
 [-17.48114267]
 [ -0.09271216]
 [-11.16980434]
 [ -1.56792242]
 [-11.66182547]
 [-10.51040627]
 [ -8.85292729]
 [-19.98957896]]
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (16, 10, 5, 5)
(120, 10, 1, 1)
x (10, 120, 9, 9)
(16, 120, 5, 5)
x (6, 10, 14, 14)
(16, 10, 10, 10)
x (10, 16, 18, 18)
(6, 16, 5, 5)
x (1, 10, 32, 32)
(6, 10, 28, 28)
x (10, 6, 36, 36)
(1, 6, 5, 5)
[[-17.57387784]
 [ -0.27352338]
 [-13.16400771]
 [-13.00328135]
 [-19.70494374]
 [ -7.

x (6, 10, 14, 14)
(16, 10, 10, 10)
x (10, 16, 18, 18)
(6, 16, 5, 5)
x (1, 10, 32, 32)
(6, 10, 28, 28)
x (10, 6, 36, 36)
(1, 6, 5, 5)
[[-11.71339824]
 [-15.34984701]
 [-11.63733537]
 [-18.47538662]
 [-13.16400771]
 [-29.41990522]
 [ -9.74163847]
 [ -6.22764936]
 [ -2.75769418]
 [-28.25565117]]
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (16, 10, 5, 5)
(120, 10, 1, 1)
x (10, 120, 9, 9)
(16, 120, 5, 5)
x (6, 10, 14, 14)
(16, 10, 10, 10)
x (10, 16, 18, 18)
(6, 16, 5, 5)
x (1, 10, 32, 32)
(6, 10, 28, 28)
x (10, 6, 36, 36)
(1, 6, 5, 5)
[[-21.01344854]
 [-24.72142296]
 [-19.70494374]
 [-12.02621616]
 [ -7.03214079]
 [-11.16980434]
 [-13.00328135]
 [ -6.86273895]
 [-17.37312804]
 [-14.71612647]]
x (10, 1, 32, 32)
(6, 1, 5, 5)
x (10, 6, 14, 14)
(16, 6, 5, 5)
x (10, 16, 5, 5)
(120, 16, 5, 5)
x (16, 10, 5, 5)
(120, 10, 1, 1)
x (10, 120, 9, 9)
(16, 120, 5, 5)
x (6, 10, 14, 14)
(16, 10, 10, 10)
x (10, 16, 18, 18)
(6, 16, 5, 5)
x (1, 10, 32, 32)


ValueError: all input arrays must have the same shape

In [ ]:
print(train_err.shape)

In [ ]:
# plot learning curves
plt.title("learning curves")
train_acc, = evaluate(cnn, accuracy, Dataloader(*train_data))
loss_curve, = plt.plot(np.mean(train_err, axis=1), 
                       label=f'train (acc: {100 * train_acc:2.2f}%)')
valid_acc, = evaluate(CNN, accuracy, Dataloader(*valid_data))
plt.plot(valid_err, linestyle='--', color=loss_curve.get_color(), 
         label=f'valid (acc: {100 * valid_acc:2.2f}%)')
plt.ylim(0, 14)  # you can change the upper limit if you want
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend();